In [1]:
import pandas as pd
import numpy as np
import json
import os
import regex
import glob
import time
from dotenv import load_dotenv, find_dotenv
import requests
import google.generativeai as genai
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path


os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()

dotenv_path = Path('/Users/chrissoria/Documents/Research/Categorization_AI_experiments/.env')
pictures_path = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files"

load_dotenv(dotenv_path=dotenv_path)

open_ai_key = os.environ.get("OPENAI_API_KEY")

In [62]:
%run /Users/chrissoria/documents/research/catllm/image_classification_functions.py

In [53]:
def process_scored_data(scored_data, pictures_path):
    processed_test_data = {}
    processed_data = {}
    for i in range(1, 5):
        pic_ty_col = f'c_79_{i}_pic_ty'
        pic_col = f'c_79_{i}_pic'
        
        # Convert to numeric with coercion
        scored_data[pic_ty_col] = pd.to_numeric(scored_data[pic_ty_col], errors='coerce')
        
        # Filter rows where pic column meets all criteria
        all_scored_data = scored_data[scored_data[pic_col].notna()]
        all_scored_data = all_scored_data[all_scored_data[pic_col] != '']
        all_scored_data = all_scored_data[all_scored_data[pic_col] != '.v']
        all_scored_data = all_scored_data[all_scored_data[pic_col] != 'i']
        all_scored_data[pic_col] = all_scored_data[pic_col].apply(lambda x: str(x) if not pd.isna(x) else '') + '.jpg'
        all_scored_data[f'{pic_col}_path'] = pictures_path + '/' + all_scored_data[pic_col]
        
        # Filter rows where pic_ty column is not NA
        scored_data_c = scored_data[scored_data[pic_ty_col].notna()]
        print(f'Initial count for {pic_ty_col}:', len(scored_data_c))
        
        # Filter out rows where pic_ty is '.v' or empty string
        scored_data_c = scored_data_c[scored_data_c[pic_ty_col] != '.v']
        print(f'After removing ".v" for {pic_ty_col}:', len(scored_data_c))
        
        scored_data_c = scored_data_c[scored_data_c[pic_ty_col] != '']
        print(f'After removing empty string for {pic_ty_col}:', len(scored_data_c))
        
        # Create paths and select columns
        scored_data_c[pic_col] = scored_data_c[pic_col].apply(lambda x: str(x) if not pd.isna(x) else '') + '.jpg'
        scored_data_c[f'{pic_col}_path'] = pictures_path + '/' + scored_data_c[pic_col]
        scored_data_c = scored_data_c[[pic_col, f'{pic_col}_path', pic_ty_col]]
        
        # Store in dictionary
        processed_test_data[f'scored_data_c_79_{i}'] = scored_data_c
        processed_data[f'all_79_{i}_scored_data'] = all_scored_data
    
    return processed_test_data, processed_data

def process_pic_column(df, pic_col, pictures_path):
    df = df[df[pic_col].notna()]
    df = df[df[pic_col] != '']
    df = df[df[pic_col] != '.v']
    df = df[df[pic_col] != '.i']
    df[pic_col] = df[pic_col].apply(lambda x: str(x) if not pd.isna(x) else '') + '.jpg'
    df[f'{pic_col}_path'] = pictures_path + '/' + df[pic_col]
    return df

read in the image scoring and filepaths

In [54]:
image_extensions = [
    '*.png', '*.jpg', '*.jpeg',
    '*.gif', '*.webp', '*.svg', '*.svgz', '*.avif', '*.apng',
    '*.tif', '*.tiff', '*.bmp',
    '*.heif', '*.heic', '*.ico',
    '*.psd'
]

image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(pictures_path, ext)))

# Sort the images by name
image_files_sorted = sorted(image_files)
print(f"Found {len(image_files_sorted)} images.")

#from all images, remove the file path and keep only the image name
image_names = [os.path.basename(image) for image in image_files_sorted]
print(image_names)

#read in ty's scoring
scored_data = pd.read_excel('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/ty_4_image_scoring_in_DR.xlsx')
more_scored_data = pd.read_excel('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/all_image_scoring_DR.xlsx')

# Dictionary to hold the processed DataFrames
processed_test_data = {}
processed_data = {}


processed_test_data, processed_data = process_scored_data(scored_data, pictures_path)
processed_data

Found 15025 images.
['1674774931963 (1).jpg', '1674774931963 (2).jpg', '1674774931963.jpg', '1674836382613 (1).jpg', '1674836382613 (2).jpg', '1674836382613.jpg', '1674836591400 (1).jpg', '1674836591400 (2).jpg', '1674836591400.jpg', '1674836668623 (1).jpg', '1674836668623 (2).jpg', '1674836668623.jpg', '1674836775244 (1).jpg', '1674836775244 (2).jpg', '1674836775244.jpg', '1674923692666 (1).jpg', '1674923692666 (2).jpg', '1674923692666.jpg', '1674924472431 (1).jpg', '1674924472431 (2).jpg', '1674924472431.jpg', '1674924847243 (1).jpg', '1674924847243 (2).jpg', '1674924847243.jpg', '1674925867953 (1).jpg', '1674925867953 (2).jpg', '1674925867953.jpg', '1674925981711 (1).jpg', '1674925981711 (2).jpg', '1674925981711.jpg', '1674926034736 (1).jpg', '1674926034736 (2).jpg', '1674926034736.jpg', '1674926048533 (1).jpg', '1674926048533.jpg', '1674926365664 (1).jpg', '1674926365664.jpg', '1674926399205 (1).jpg', '1674926399205.jpg', '1674926425777 (1).jpg', '1674926425777.jpg', '1675110452866

{'all_79_1_scored_data':           pid    hhid                        globalrecordid  c_interid  \
 0    16002901  160029  e0db6557-4720-4822-a1ab-85e8d96f0faa          1   
 1    14404901  144049  dd3ae28f-e7e9-4352-9d4d-54ec145b2057          1   
 2    15404202  154042  149524b7-4d68-4576-998b-8523b9ac5c70          1   
 3    11709801  117098  707daf71-a4cd-478a-981b-5988eca2de0f          1   
 4    14107301  141073  26366b9d-5a6f-4c94-8da1-23476e5a9fb9          1   
 ..        ...     ...                                   ...        ...   
 468  14008601  140086  a2eea739-f7d6-49ad-9a8b-65edd0a5496b          5   
 469  15700302  157003  52304312-b35e-4eaf-b276-572654a360af          5   
 470  15702302  157023  48f40f09-7c1e-4e12-af5d-6effed5e92fc          5   
 471  16002902  160029  6899a1ff-9573-4aff-a41b-61180b25e5d5          5   
 472  17308001  173080  e4281c65-2cf1-4325-bf79-f0fe96a936aa         80   
 
         c_72_1_pic c_72_1_pic_ty     c_72_2_pic c_72_2_pic_ty     c_72_3_

next, pull out the list of pictures in the all_images folder that I want to score

In [81]:
# process the columns
more_scored_data_c_79_1 = process_pic_column(more_scored_data, 'c_79_1_pic', pictures_path)
more_scored_data_c_79_2 = process_pic_column(more_scored_data, 'c_79_2_pic', pictures_path)
more_scored_data_c_79_3 = process_pic_column(more_scored_data, 'c_79_3_pic', pictures_path)
more_scored_data_c_79_4 = process_pic_column(more_scored_data, 'c_79_4_pic', pictures_path)

more_scored_data_c_72_1 = process_pic_column(more_scored_data, 'c_72_1_pic', pictures_path)
more_scored_data_c_72_2 = process_pic_column(more_scored_data, 'c_72_2_pic', pictures_path)
more_scored_data_c_72_3 = process_pic_column(more_scored_data, 'c_72_3_pic', pictures_path)
more_scored_data_c_72_4 = process_pic_column(more_scored_data, 'c_72_4_pic', pictures_path)

more_scored_data_c_79_4

,pid,hhid,globalrecordid,c_interid,c_72_1_pic,c_72_1_pic_ty,c_72_2_pic,c_72_2_pic_ty,c_72_3_pic,c_72_3_pic_ty,...,c_79_4_pic_found,cs_72_1,cs_72_2,cs_72_3,cs_72_4,cs_79_1,cs_79_2,cs_79_3,cs_79_4,c_79_4_pic_path
0,11603002,116030.0,46741e3d-7ff0-4ba4-a99c-585e178b0b70,1.0,1720117790904,NaN,1720117832907,NaN,1720117819502,NaN,...,found,1.0,3.0,1.0,3.0,1.0,0.0,2.0,2.0,/Users/chrissoria/Google Drive/other computers...
1,15802902,158029.0,d932b9d8-6504-4473-b079-ef090166b5ad,1.0,1724250315558,NaN,1724250426251,NaN,1724250406698,NaN,...,found,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
2,12807202,128072.0,19a96d02-7ac2-46d6-8375-bc12a52cd540,1.0,1717601448066,NaN,1717601375530,NaN,1717601424408,NaN,...,found,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
3,13406701,134067.0,31e971b4-ad1a-42eb-919e-1bab5ee452c0,1.0,1711631266376,NaN,1711631208851,NaN,1711631228388,NaN,...,found,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
4,13001702,130017.0,ff4fc94a-6019-482b-9690-58d821150931,1.0,1718640688577,NaN,1718640625894,NaN,1718640654873,NaN,...,found,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,/Users/chrissoria/Google Drive/other computers...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,15603501,156035.0,29a86825-ff0d-4a61-b9c6-a842871b1e3e,5.0,1730827065617,NaN,1730827205903,NaN,1730827232863,NaN,...,found,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
1210,15803401,158034.0,a7a0f343-cb8a-43e9-9acb-89ddddf38f34,5.0,1724381924916,2,1724381974069,2,1724382022321,2,...,found,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
1211,15702302,157023.0,48f40f09-7c1e-4e12-af5d-6effed5e92fc,5.0,1726183412629,2,1726183543681,1,1726183631116,1,...,found,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,/Users/chrissoria/Google Drive/other computers...
1216,10911801,109118.0,b1305cb2-a90c-4c93-a4db-5392bb0ad5fd,6.0,.i,NaN,.i,NaN,.i,NaN,...,found,2.0,2.0,1.0,0.0,2.0,0.0,1.0,0.0,/Users/chrissoria/Google Drive/other computers...


In [ ]:
# next, find the images that are in the more_scored_data but not in processed_data
more_scored_data_c_79_1['c_79_1_already_scored'] = more_scored_data_c_79_1['c_79_1_pic_path'].isin(processed_data['all_79_1_scored_data']['c_79_1_pic_path'])
more_scored_data_c_79_2['c_79_2_already_scored'] = more_scored_data_c_79_2['c_79_2_pic_path'].isin(processed_data['all_79_2_scored_data']['c_79_2_pic_path'])
more_scored_data_c_79_3['c_79_3_already_scored'] = more_scored_data_c_79_3['c_79_3_pic_path'].isin(processed_data['all_79_3_scored_data']['c_79_3_pic_path'])
more_scored_data_c_79_4['c_79_4_already_scored'] = more_scored_data_c_79_4['c_79_4_pic_path'].isin(processed_data['all_79_4_scored_data']['c_79_4_pic_path'])

#count the number of images that are already scored
print('Number of images already scored in c_79_1:', more_scored_data_c_79_1['c_79_1_already_scored'].sum())
print('Number of images already scored in c_79_2:', more_scored_data_c_79_2['c_79_2_already_scored'].sum())
print('Number of images already scored in c_79_3:', more_scored_data_c_79_3['c_79_3_already_scored'].sum())
print('Number of images already scored in c_79_4:', more_scored_data_c_79_4['c_79_4_already_scored'].sum())

Number of images already scored in c_79_1: 480
Number of images already scored in c_79_2: 480
Number of images already scored in c_79_3: 480
Number of images already scored in c_79_4: 480


In [ ]:
more_scored_data_c_79_1 = more_scored_data_c_79_1[more_scored_data_c_79_1['c_79_1_already_scored'] == False].reset_index(drop=True)
more_scored_data_c_79_2 = more_scored_data_c_79_2[more_scored_data_c_79_2['c_79_2_already_scored'] == False].reset_index(drop=True)
more_scored_data_c_79_3 = more_scored_data_c_79_3[more_scored_data_c_79_3['c_79_3_already_scored'] == False].reset_index(drop=True)
more_scored_data_c_79_4 = more_scored_data_c_79_4[more_scored_data_c_79_4['c_79_4_already_scored'] == False].reset_index(drop=True)

715
235


In [ ]:
image_categories = ["It has a drawing of a circle",
                    "The circle is closed",
                    "The circle is almost closed",
                    "The circle is circular",
                    "The circle is almost circular",
                    "None of the above"]

c_79_1 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a circle", 
    image_input= processed_data['all_79_1_scored_data']['c_79_1_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_1_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

Next, let's score the remaining images

In [98]:
image_categories = ["It has a drawing of a circle",
                    "The circle is closed",
                    "The circle is almost closed",
                    "The circle is circular",
                    "The circle is almost circular",
                    "None of the above"]

c_79_1_next = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a circle", 
    image_input= more_scored_data_c_79_1['c_79_1_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_1_machine_scored_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

Provided a list of 235 images.
Categories to classify:
1. It has a drawing of a circle
2. The circle is closed
3. The circle is almost closed
4. The circle is circular
5. The circle is almost circular
6. None of the above


Categorising images:   0%|          | 0/235 [00:00<?, ?it/s]

Categorising images: 100%|██████████| 235/235 [31:51<00:00,  8.13s/it] 


In [ ]:
image_categories = ["It has a drawing of a diamond",
                    "The diamond has 4 sides",
                    "The diamond looks like a diamond",
                    "All sides of the diamond are about equal",
                    "It looks more like a square than a diamond",
                    "None of the above"]

c_79_2 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a diamond", 
    image_input= processed_data['all_79_2_scored_data']['c_79_2_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_2_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

In [99]:
image_categories = ["It has a drawing of a diamond",
                    "The diamond has 4 sides",
                    "The diamond looks like a diamond",
                    "All sides of the diamond are about equal",
                    "It looks more like a square than a diamond",
                    "None of the above"]

c_79_2 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a diamond", 
    image_input= more_scored_data_c_79_2['c_79_2_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_2_machine_scored_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

Provided a list of 139 images.
Categories to classify:
1. It has a drawing of a diamond
2. The diamond has 4 sides
3. The diamond looks like a diamond
4. All sides of the diamond are about equal
5. It looks more like a square than a diamond
6. None of the above


Categorising images: 100%|██████████| 139/139 [19:00<00:00,  8.21s/it]


In [ ]:
image_categories = ["It has a drawing of a diamond",
                    "The diamond has 4 sides",
                    "The diamond looks like a diamond",
                    "All sides of the diamond are about equal",
                    "It looks more like a square than a diamond",
                    "None of the above"]

c_79_2 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a diamond", 
    image_input= processed_data['all_79_2_scored_data'][260:473]['c_79_2_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_2_machine_scored_continued.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

In [ ]:
image_categories = ["It has a drawing of overlapping rectangles",
                    "Rectangle 1 has 4 sides",
                    "Rectangle 2 has 4 sides",
                    "The rectangles are overlapping",
                    "The rectangles overlap contains a longer vertical rectangle with top and bottom portruding",
                    "None of the above"]

c_79_3 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a overlapping rectangles", 
    image_input= processed_data['all_79_3_scored_data']['c_79_3_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_3_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

In [100]:
image_categories = ["It has a drawing of overlapping rectangles",
                    "Rectangle 1 has 4 sides",
                    "Rectangle 2 has 4 sides",
                    "The rectangles are overlapping",
                    "The rectangles overlap contains a longer vertical rectangle with top and bottom portruding",
                    "None of the above"]

c_79_3 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a overlapping rectangles", 
    image_input= more_scored_data_c_79_2['c_79_2_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_3_machine_scored_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

Provided a list of 139 images.
Categories to classify:
1. It has a drawing of overlapping rectangles
2. Rectangle 1 has 4 sides
3. Rectangle 2 has 4 sides
4. The rectangles are overlapping
5. The rectangles overlap contains a longer vertical rectangle with top and bottom portruding
6. None of the above


Categorising images: 100%|██████████| 139/139 [12:12<00:00,  5.27s/it]


In [ ]:
image_categories = ["It has a drawing of a cube",
                    "Front face properly oriented",
                    "The front of the cube is visible",
                    "The internal lines drawn correctly in a way that makes it look 3D",
                    "Opposite front and back sides are parallel",
                    "None of the above"]

c_79_4 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a cube", 
    image_input= processed_data['all_79_4_scored_data']['c_79_4_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_4_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

In [ ]:
open_ai_key = os.environ.get("OPENAI_API_KEY")
print(open_ai_key)

In [ ]:
image_categories = ["It has a drawing of a cube",
                    "Front face properly oriented",
                    "The front of the cube is visible",
                    "The internal lines drawn correctly in a way that makes it look 3D",
                    "Opposite front and back sides are parallel",
                    "None of the above"]

c_79_4 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a cube", 
    image_input= processed_data['all_79_4_scored_data'][396:473]['c_79_4_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_4_machine_scored_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

In [101]:
image_categories = ["It has a drawing of a cube",
                    "Front face properly oriented",
                    "The front of the cube is visible",
                    "The internal lines drawn correctly in a way that makes it look 3D",
                    "Opposite front and back sides are parallel",
                    "None of the above"]

c_79_4 = llm_extract_image_multi_class(
    image_description = "this image contains within it a drawing of a cube", 
    image_input= more_scored_data_c_79_4['c_79_4_pic_path'].tolist(), 
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_79_4_machine_scored_next_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

Provided a list of 361 images.
Categories to classify:
1. It has a drawing of a cube
2. Front face properly oriented
3. The front of the cube is visible
4. The internal lines drawn correctly in a way that makes it look 3D
5. Opposite front and back sides are parallel
6. None of the above


Categorising images: 100%|██████████| 361/361 [9:44:49<00:00, 97.20s/it]     


In [61]:
c_79_4_image_score = llm_extract_image_score(
    reference_image_description = "this image contains within it a drawing of a cube",
    image_input= more_scored_data_c_79_4['c_79_4_pic_path'].tolist(),
    reference_image = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/reference_cube.png",
    api_key = os.environ.get("OPENAI_API_KEY"),
    safety = True,
    to_csv = True,
    filename="c_79_4_machine_scored_image.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/"
)

Directory /Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/ exists
Provided a list of 841 images.


Categorising images:  24%|██▍       | 202/841 [28:14<1:29:19,  8.39s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/1720895438199.jpg'

merge all the columns into one

In [93]:
image_categories = ["It has a drawing of a circle next to a reference",
                    "The drawing is not similar to the reference (regardless of size)",
                    "The drawing is similar to the reference (regardless of size)",
                    "The drawing of a circle is closed",
                    "The drawing of a circle is almost closed",
                    "The drawing of a circle is circular",
                    "The drawing of a circle is almost circular",
                    "The drawing is not at all circular",
                    "None of the above"]

c_72_1 = llm_extract_image_multi_class(
    image_description = "This image reference image of a circle on the top left. The drawing is meant to be similar to the reference circle.", 
    image_input= more_scored_data_c_72_1['c_72_1_pic_path'].tolist(),
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_72_1_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

c_72_1 = c_72_1.rename(columns={
    "1":"drawing_present",
    "2":"not_similar",
    "3":"similar",
    "4": "cir_closed",
    "5": "cir_almost_closed",
    "6": "cir_round",
    "7": "cir_almost_round",
    "8": "cir_not_round",
    "9": "none"
})

c_72_1.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_1_machine_scored_full.csv', index=False)

Provided a list of 855 images.
Categories to classify:
1. It has a drawing of a circle next to a reference
2. The drawing is not similar to the reference (regardless of size)
3. The drawing is similar to the reference (regardless of size)
4. The drawing of a circle is closed
5. The drawing of a circle is almost closed
6. The drawing of a circle is circular
7. The drawing of a circle is almost circular
8. The drawing is not at all circular
9. None of the above


Categorising images: 100%|██████████| 855/855 [2:18:38<00:00,  9.73s/it]  


In [95]:
image_categories = ["It has a drawing of a diamond next to a reference",
                    "The drawing is not similar to the reference",
                    "The drawing is similar to the reference (regardless of size)",
                    "The drawing of a diamond has 4 sides",
                    "The drawing of a diamond has 4 sides that are about equal",
                    "The drawing of a diamond looks more like a square",
                    "None of the above"]

c_72_2 = llm_extract_image_multi_class(
    image_description = "This image reference image of a diamond on the top left. The drawing is meant to be similar to the reference diamond.", 
    image_input= more_scored_data_c_72_2['c_72_2_pic_path'].tolist(),
    categories = image_categories,
    safety = True,
    to_csv = True,
    filename="c_72_2_machine_scored.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/",
    api_key= os.environ.get("OPENAI_API_KEY"))

c_72_2 = c_72_2.rename(columns={
    "1": "drawing_present",
    "2": "not_similar",
    "3": "similar",
    "4": "diamond_4_sides",
    "5": "diamond_equal_sides",
    "6": "diamond_square",
    "7": "none"
})

c_72_2.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_2_machine_scored_full.csv', index=False)

Provided a list of 848 images.
Categories to classify:
1. It has a drawing of a diamond next to a reference
2. The drawing is not similar to the reference
3. The drawing is similar to the reference (regardless of size)
4. The drawing of a diamond has 4 sides
5. The drawing of a diamond has 4 sides that are about equal
6. The drawing of a diamond looks more like a square
7. None of the above


Categorising images: 100%|██████████| 848/848 [3:43:59<00:00, 15.85s/it]      


In [ ]:
c_79_4_image_score = llm_extract_image_score(
    reference_image_description = "this image contains within it a drawing of a cube",
    image_input= more_scored_data_c_79_4['c_79_4_pic_path'][202:841].tolist(),
    reference_image = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/reference_cube.png",
    api_key = os.environ.get("OPENAI_API_KEY"),
    safety = True,
    to_csv = True,
    filename="c_79_4_machine_scored_image_next.csv",
    save_directory = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/"
)

In [12]:
#concat the two on top of each other
c_79_4_image_score_first = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_4_machine_scored_image.csv')
c_79_4_image_score_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_4_machine_scored_image_next.csv')

c_79_4_image_score = pd.concat([c_79_4_image_score_first, c_79_4_image_score_next], ignore_index=True)

c_79_4_image_score = c_79_4_image_score[['image_input', 'score']].rename(columns={
    'score':'quality_score'
})

c_79_4_image_score

,image_input,quality_score
0,/Users/chrissoria/Google Drive/other computers...,4.0
1,/Users/chrissoria/Google Drive/other computers...,1.0
2,/Users/chrissoria/Google Drive/other computers...,3.0
3,/Users/chrissoria/Google Drive/other computers...,1.0
4,/Users/chrissoria/Google Drive/other computers...,1.0
...,...,...
836,/Users/chrissoria/Google Drive/other computers...,2.0
837,/Users/chrissoria/Google Drive/other computers...,2.0
838,/Users/chrissoria/Google Drive/other computers...,2.0
839,/Users/chrissoria/Google Drive/other computers...,1.0


In [13]:
c_79_1_first = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_1_machine_scored.csv')
c_79_1_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_1_machine_scored_next.csv')

#stack both on top of each other
c_79_1 = pd.concat([c_79_1_first, c_79_1_next], ignore_index=True)
c_79_1['image_filepath'] = c_79_1['image_input']
c_79_1['image_input'] = c_79_1['image_input'].str.lstrip('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_1 = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_72_1_machine_scored.csv')

# for c_79_2 
c_79_2_first = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_2_machine_scored.csv')
c_79_2_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_2_machine_scored_next.csv')
c_79_2_continued = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_2_machine_scored_continued.csv')
#stack all three on top of each other
c_79_2 = pd.concat([c_79_2_first, c_79_2_next, c_79_2_continued], ignore_index=True)
c_79_2['image_filepath'] = c_79_2['image_input']
c_79_2['image_input'] = c_79_2['image_input'].str.lstrip('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_2 = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_72_2_machine_scored.csv')

# for c_79_3
c_79_3_first = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_3_machine_scored.csv')
c_79_3_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_3_machine_scored_next.csv')
#stack both on top of each other
c_79_3 = pd.concat([c_79_3_first, c_79_3_next], ignore_index=True)
c_79_3['image_filepath'] = c_79_3['image_input']
c_79_3['image_input'] = c_79_3['image_input'].str.lstrip('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_3 = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_72_3_machine_scored.csv')

# for c_79_4
c_79_4_first = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_4_machine_scored.csv')
c_79_4_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_4_machine_scored_next.csv')
c_79_4_next_next = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_79_4_machine_scored_next_next.csv')
#stack all three on top of each other
c_79_4 = pd.concat([c_79_4_first, c_79_4_next, c_79_4_next_next], ignore_index=True)
c_79_4 = pd.merge(c_79_4, c_79_4_image_score, how='left',on='image_input')
c_79_4['image_filepath'] = c_79_4['image_input']
c_79_4['image_input'] = c_79_4['image_input'].str.lstrip('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_4 = pd.read_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/old_tries/c_72_4_machine_scored.csv')

#save all three to csv
c_79_1.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_1_machine_scored_full.csv', index=False)
c_79_2.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_2_machine_scored_full.csv', index=False)
c_79_3.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_3_machine_scored_full.csv', index=False)
c_79_4.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_4_machine_scored_full.csv', index=False)

producing scores

In [17]:
c_79_1 = c_79_1.rename(columns={
    "1":"drawing_present",
    "2": "cir_closed",
    "3": "cir_almost_closed",
    "4": "cir_round",
    "5": "cir_almost_round",
    "6": "none"
})

c_79_1['score'] = c_79_1['cir_almost_closed'] + c_79_1['cir_almost_round'] + c_79_1['cir_round']
c_79_1['score'] = np.where(c_79_1['none'] == 1, 0, c_79_1['score'])

c_79_1['score_strict'] = c_79_1['cir_almost_closed'] + c_79_1['cir_round']
c_79_1['score_strict'] = np.where(c_79_1['none'] == 1, 0, c_79_1['score_strict'])

c_79_1 = c_79_1.rename(columns={
    'image_input' : 'c_79_1_pic',
    'score': 'c_79_1_score_AI',
    'score_strict': 'c_79_1_score_strict_AI'
})

c_79_1['c_79_1_pic'] = c_79_1['image_filepath'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_79_1.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_1_machine_scored_full.csv', index=False)

c_79_1_score = c_79_1[['c_79_1_pic', 'c_79_1_score_AI', 'c_79_1_score_strict_AI']]

c_79_1_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_1_machine_score.csv', index=False)

In [18]:
c_79_2 = c_79_2.rename(columns={
    "1":"drawing_present",
    "2": "diamond_4_sides",
    "3": "diamond_looks_like_diamond",
    "4": "diamond_equal_sides",
    "5": "diamond_square",
    "6": "none"
})

c_79_2['score'] = c_79_2['diamond_4_sides'] + c_79_2['diamond_equal_sides'] + c_79_2['diamond_looks_like_diamond'] - c_79_2['diamond_square']
c_79_2['score'] = np.where(c_79_2['none'] == 1, 0, c_79_2['score'])
c_79_2['score'] = np.where(c_79_2['score'] < 0, 0, c_79_2['score'])

c_79_2 = c_79_2.rename(columns={
    'image_input' : 'c_79_2_pic',
    'score': 'c_79_2_AI_score'
})

c_79_2['c_79_2_pic'] = c_79_2['image_filepath'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_79_2.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_2_machine_scored_full.csv', index=False)

c_79_2_score = c_79_2[['c_79_2_pic', 'c_79_2_AI_score']]
c_79_2_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_2_machine_score.csv', index=False)


In [19]:
c_79_3 = c_79_3.rename(columns={
    "1":"drawing_present",
    "2": "r1_4_sides",
    "3": "r2_4_sides",
    "4": "rectangles_overlap",
    "5": "rectangles_cross",
    "6": "none"
})

c_79_3['score'] = np.where((c_79_3['r1_4_sides'] == 1) & (c_79_3['r2_4_sides'] == 1), 1, 0)
c_79_3['score'] = np.where((c_79_3['rectangles_overlap'] == 1) & (c_79_3['rectangles_cross']), c_79_3['score']+1, c_79_3['score'])
c_79_3['score'] = np.where(c_79_3['none'] == 1, 0, c_79_3['score'])

c_79_3 = c_79_3.rename(columns={
    'image_input' : 'c_79_3_pic',
    'score': 'c_79_3_AI_score'
})

c_79_3['c_79_3_pic'] = c_79_3['image_filepath'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')


c_79_3.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_3_machine_scored_full.csv', index=False)

c_79_3_score = c_79_3[['c_79_3_pic', 'c_79_3_AI_score']]
c_79_3_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_3_machine_score.csv', index=False)

In [22]:
c_79_4 = c_79_4.rename(columns={
    "1":"drawing_present",
    "2": "cube_front_face",
    "3": "cube_front_visible",
    "4": "cube_internal_lines",
    "5": "cube_opposite_sides",
    "6": "none"
})

c_79_4['score'] = c_79_4['cube_front_visible'] + c_79_4['cube_internal_lines'] + c_79_4['cube_opposite_sides']
c_79_4['score'] = np.where(c_79_4['none'] == 1, 0, c_79_4['score'])
c_79_4['score'] = np.where(c_79_4['quality_score'] == 0, 0, c_79_4['score'])
c_79_4['score'] = np.where(c_79_4['quality_score'] > 2, c_79_4['score']+1, c_79_4['score'])

c_79_4 = c_79_4.rename(columns={
    'image_input' : 'c_79_4_pic',
    'score': 'c_79_4_AI_score'
})

c_79_4['c_79_4_pic'] = c_79_4['image_filepath'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_79_4.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_4_machine_scored_full.csv', index=False)

c_79_4_score = c_79_4[['c_79_4_pic', 'c_79_4_AI_score']]

c_79_4_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_79_4_machine_score.csv', index=False)

In [23]:
c_72_1

c_72_1 = c_72_1.rename(columns={
    "1":"drawing_present",
    "2":"not_similar",
    "3":"similar",
    "4": "cir_closed",
    "5": "cir_almost_closed",
    "6": "cir_round",
    "7": "cir_almost_round",
    "8": "cir_not_round",
    "9": "none"
})

c_72_1.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_1_machine_scored_full.csv', index=False)

In [24]:
c_72_1['c_72_1_AI_score'] = c_72_1['cir_closed'] + c_72_1['cir_almost_closed'] + c_72_1['cir_almost_round'] + c_72_1['cir_round']
c_72_1['c_72_1_AI_score'] = np.where(c_72_1['none'] == 1, 0, c_72_1['c_72_1_AI_score'])
c_72_1['c_72_1_AI_score'] = np.where((c_72_1['not_similar'] == 1) & (c_72_1['c_72_1_AI_score'] == 0), 0, c_72_1['c_72_1_AI_score'])

c_72_1['c_72_1_AI_score_strict'] = np.where(c_72_1['not_similar'] == 1, c_72_1['c_72_1_AI_score']-1, c_72_1['c_72_1_AI_score'])
c_72_1['c_72_1_AI_score_strict'] = np.where(c_72_1['c_72_1_AI_score_strict'] == -1, 0, c_72_1['c_72_1_AI_score_strict'])

c_72_1['c_72_1_pic'] = c_72_1['image_input'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_1.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_1_machine_scored_full.csv', index=False)

c_72_1_score = c_72_1[['c_72_1_pic', 'c_72_1_AI_score','c_72_1_AI_score_strict']]

c_72_1_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_1_machine_score.csv', index=False)


In [25]:
c_72_2 = c_72_2.rename(columns={
    "1": "drawing_present",
    "2": "not_similar",
    "3": "similar",
    "4": "diamond_4_sides",
    "5": "diamond_equal_sides",
    "6": "diamond_square",
    "7": "none"
})

c_72_2.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_2_machine_scored_full.csv', index=False)

In [26]:
c_72_2['c_72_2_AI_score'] = c_72_2['diamond_4_sides'] + c_72_2['diamond_equal_sides']
c_72_2['c_72_2_AI_score'] = np.where(c_72_2['none'] == 1, 0, c_72_2['c_72_2_AI_score'])
c_72_2['c_72_2_AI_score'] = np.where(c_72_2['drawing_present'] == 0, 0, c_72_2['c_72_2_AI_score'])
c_72_2['c_72_2_AI_score'] = np.where((c_72_2['not_similar'] == 1) & (c_72_2['c_72_2_AI_score'] == 0), 0, c_72_2['c_72_2_AI_score'])
c_72_2['c_72_2_AI_score'] = np.where(c_72_2['c_72_2_AI_score'] < 0, 0, c_72_2['c_72_2_AI_score'])

c_72_2['c_72_2_AI_score_strict'] = np.where(c_72_2['diamond_square'] == 1, c_72_2['c_72_2_AI_score']-1, c_72_2['c_72_2_AI_score'])
c_72_2['c_72_2_AI_score_strict'] = np.where(c_72_2['c_72_2_AI_score_strict'] <0, 0, c_72_2['c_72_2_AI_score_strict'])

c_72_2['c_72_2_pic'] = c_72_2['image_input'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_2.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_2_machine_scored_full.csv', index=False)

c_72_2_score = c_72_2[['c_72_2_pic', 'c_72_2_AI_score','c_72_2_AI_score_strict']]
c_72_2_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_2_machine_score.csv', index=False)


In [27]:
c_72_3['c_72_3_AI_score'] = np.where((c_72_3['rect_1_4_sides'] == 1) & (c_72_3['rect_2_4_sides'] == 1), 1, 0)
c_72_3['c_72_3_AI_score'] = np.where((c_72_3['rectangles_overlap'] == 1), c_72_3['c_72_3_AI_score'] + 1, c_72_3['c_72_3_AI_score'])

c_72_3['c_72_3_AI_score'] = np.where(c_72_3['none'] == 1, 0, c_72_3['c_72_3_AI_score'])

c_72_3['c_72_3_AI_score'] = np.where(c_72_3['not_similar'] == 1, c_72_3['c_72_3_AI_score']-1, c_72_3['c_72_3_AI_score'])

c_72_3['c_72_3_pic'] = c_72_3['image_input'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_3.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_3_machine_scored_full.csv', index=False)

c_72_3_score = c_72_3[['c_72_3_pic', 'c_72_3_AI_score']]

c_72_3_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_3_machine_score.csv', index=False)

In [28]:
c_72_4['c_72_4_AI_score'] = c_72_4['cube_front_face'] + c_72_4['cube_internal_lines'] + c_72_4['cube_opposite_sides']
c_72_4['c_72_4_AI_score'] = np.where(c_72_4['none'] == 1, 0, c_72_4['c_72_4_AI_score'])

c_72_4['c_72_4_AI_score'] = np.where((c_72_4['drawing_present'] == 0) & (c_72_4['c_72_4_AI_score'] == 0), 0, c_72_4['c_72_4_AI_score'])
c_72_4['c_72_4_AI_score'] = np.where((c_72_4['not_similar'] == 1) & (c_72_4['c_72_4_AI_score'] == 0), 0, c_72_4['c_72_4_AI_score'])

c_72_4['c_72_4_pic'] = c_72_4['image_input'].str.removeprefix('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/All_Image_Files/')

c_72_4.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_4_machine_scored_full.csv', index=False)

c_72_4_score = c_72_4[['c_72_4_pic', 'c_72_4_AI_score']]

c_72_4_score.to_csv('/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom/Figures/c_72_4_machine_score.csv', index=False)